# **Confidence interval estimation**

by Felix Fritzen (fritzen@simtech.uni-stuttgart.de)

additional material for the course _Data processing for engineers and scientists_ at the University of Stuttgart


## **Objective**
Given $n$ data samples stored in a vector $\underline{y}\in\mathbb{R}^n$ of samples drawn from IID random variables $Y_1, \dots, Y_n$. We are interested in a **quantity of interest (QOI)** of the underlying i.i.d. random variables  $Y_1, Y_2, \dots, Y_n$ with standard deviation $\sigma_y$.

**Point estimators** provide single values for the QOI, for instance **maximum likelihood estimation (MLE)**. However, they <font color='red'>**do not**</font> provide **information about the range of the QOI**.

A **confidence interval** $\mathcal{I}(p)\subseteq \mathbb{R}$ is linked to a confidence level $p\in [0,1]$ which expresses the probability of $\mathcal{I}(p)$ to contain the true QOI.

## **Outline**
- Confidence interval of the **expectation** using the Central Limit Theorem (for large $n$) and Student's $t$-distribution (for low and moderate $n$)
- Confidence interval of the **variance** using the $\chi^2$ distribution


In [1]:
import numpy as np
# import functions for accessing Student's t-distribution and the normal distribution
from scipy.stats import t as t_dist
from scipy.stats import norm as norm_dist

## **Confidence interval estimation for $E(Y)$**
The MLE estimator for the expectation is the sample mean $$\overline{y}=\frac{1}{n} \sum_{i=1}^n y_i.$$
Two cases must be distinguished for interval estimation:
- low to moderate number of samples (in general $n<30$ for a single RV)
- high sample count ($n>30$).

### **High sample count** ($n>30$)
The Central Limit Theorem can be assumed to hold if a sufficient number of samples is available. Then
$$\overline{y} \sim \mathcal{N} \left( E(Y_1), \frac{\sigma_y^2}{n} \right), \qquad \sigma_y^2 \approx \sigma^2 = \frac{1}{n-1} \sum_{i=1}^n ( y_i - \overline{y}_i )^2$$
Using the inverse distribution function $F^{-1}(p)$ of the standard normal distribution one obtains
$$\Delta = F^{-1} \left( \frac{1+p}{2} \right) \frac{\sigma_n}{\sqrt{n}} \qquad \mathcal{I}(p) =  \left( \overline{y}_n-\Delta, \overline{y}_n+\Delta \right]$$

**Example 1: uniformly random samples**

In [10]:
n       = 10
LB      = np.random.uniform(-1,1, size=1)
UB      = np.random.uniform(1, 5, size=1) + LB

# theoretical results for the underlying distribution:
sigma_y = np.sqrt( (UB-LB)**2/12 )
EY      = (UB+LB)/2

# generate random data
y       = np.random.uniform( LB, UB, size=n )

# estimate ybar and sigma
ybar    = np.sum(y)/n
sigma   = np.sqrt(np.sum((y-ybar)**2)/(n-1))
sigma_ybar = sigma / np.sqrt(n)

# confidence level
p       = 0.925
q       = (1.0+p)/2.0

Delta   = norm_dist.ppf(q) * sigma_ybar

print('range for data .................. [%5.2f, %5.2f]' % (LB, UB))
print('number of samples ............... %10d' % n)
print('ybar ............................ %10.5f' % ybar )
print('E(y) (true value) ............... %10.5f' % (EY) )
print('sigma_y (analytical) ............ %10.5f' % ( sigma_y ) )
print('sigma_n (estimated) ............. %10.5f' % sigma )
print('sigma_ybar ...................... %10.5f' % sigma_ybar )
print('confidence level ................ %10.5f' % p )
print('Delta ........................... %10.5f   (= %8.5f * sigma_ybar = %8.5f * sigma)' % ( Delta, Delta/sigma_ybar, Delta/sigma) )


range for data .................. [-0.20,  1.75]
number of samples ...............         10
ybar ............................    0.94127
E(y) (true value) ...............    0.77345
sigma_y (analytical) ............    0.56148
sigma_n (estimated) .............    0.74063
sigma_ybar ......................    0.23421
confidence level ................    0.92500
Delta ...........................    0.41700   (=  1.78046 * sigma_ybar =  0.56303 * sigma)


*Remarks*
- the width $w=2\Delta$ of $\mathcal{I}(p)$ scales with $\sigma_y/\sqrt{n}$
- the estimator $\sigma$ for $\sigma_y$ should be unbiased (although the difference to the MLE estimator is negligible)

### **Low/moderate sample counts $(n \leq 30)$**

For lower sampel counts Student's $t$-distribution is used. We set $\nu=n-1$ and obtain
$$\Delta = F_t^{-1} \left( \frac{1+p}{2} \, \Big\vert \, \nu \right) \frac{\sigma_n}{\sqrt{n}}$$

In [ ]:
nu = n-1
Delta_t = t_dist(nu).ppf(q) * sigma_n/np.sqrt(n)
print('Delta (CLT) vs. Delta (Student\'s t-distribution): %10.5f  /  %10.5f  /  ratio: %10.5f ' \
      % ( Delta, Delta_t, Delta/Delta_t ))

**Observations**
- $\Delta < \Delta_t$
- $\underset{n\to \infty}{\text{lim}} \Delta_t-\Delta \to 0$
- the higher $p$, the bigger the difference in $\Delta$ and $\Delta_t$ *(but small quantitative difference)*
- ratios for some $n$:

In [ ]:
N = np.array([5,10,30,100])
P = np.array([0.9,0.925,0.95,0.975,0.995])
for p in P:
    q=(1+p)/2
    print('confidence level:   %6.3f%%' % (100*p) )
    for n in N:
        print('n = %3d  --> ratio:  %6.4f' % ( n, norm_dist.ppf(q)/t_dist(n-1).ppf(q)))